# **교차 검증(Cross Validation)**

- 테스트 세트를 사용하지 않고 훈련 세트를 나누어 **검증 세트**로 모델을 평가하는 것
- 테스트하고 싶은 매개변수를 바꾸어 가며 가장 좋은 모델을 선택
- 선택된 매개변수로 전체 훈련 데이터에서 모델을 다시 훈련
- 테스트 세트에서 최종 점수 평가

- 교차 검증 : **검증 세트를 K개의 폴드(덩어리)로 떼어내어 평가하는 과정을 여러번 반복**
- 이 점수를 평균하여 최종 검증 점수를 얻어낸다
- sklearn.model_selection의 cross_validate 사용

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
import numpy as np
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 검증 세트 분리
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
# 결정 트리 학습
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
# 과적합 상태
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## **교차 검증 실행**

- 기본적으로 5폴드 교차 검증 시행
  - cv로 지정할 수 있음
- fit_time : 모델 훈련 시간
- score_time : 검증하는 시간
- test_score : 테스트 점수
- 최종 점수는 test_score를 평균하여 얻을 수 있다

분할기(splitter)
- cross_validate는 훈련 세트를 섞어서 폴드를 나누지 않는다
- train_test_split으로 전체 데이터를 섞어서 따로 섞지 않았지만 교차검증 시 분할기(splitter)를 지정해야 함
  - 회귀 모델일 경우 일반적으로 KFold
  - 분류 모델일 경우 StratifiedKFold

In [7]:
from sklearn.model_selection import cross_validate
# 평가할 모델 객체, 훈련 세트 전체를 함수에 전달
scores = cross_validate(dt, train_input, train_target)
# cross_val_score()는 test_score 값만 반환
print(scores)

{'fit_time': array([0.01457286, 0.01018357, 0.01044297, 0.0108285 , 0.00984883]), 'score_time': array([0.00173688, 0.00134587, 0.00130963, 0.0014689 , 0.00135565]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
# 훈련 세트 분할하기
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 훈련 세트를 섞은 후 교차 검증을 수행하고 싶다면
# n_splits는 몇 개의 폴드로 교차 검증을 할지 정함
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## **문제 정의**

- 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델 찾기
- 테스트 세트를 사용하지 않고 교차 검증을 통해 고르기

## **그리드 서치(Grid Search)**

- 하이퍼파라미터를 튜닝하는 과정을 도와주는 모듈
- 한 매개변수의 최적의 값을 찾고 다른 매개변수의 최적의 값을 찾는 건 불가능
- 그럼 어떻게 하지?
  - 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 함 -> 그리드 서치 이용
- 그리드 서치는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
- 훈련이 끝나면 자동으로 최적의 파라미터 조합을 이용하여 전체 훈련 세트로 다시 훈련함
  - 최고 모델은 best_estimator_ 속성에 저장됨
  - 최적의 매개변수는 best_params_ 속성에 저장됨
- GridSearchCV에 존재

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
# cv의 기본값은 5이므로 min_impurity_decrease값마다 5-폴드 교차 검증을 수행 (5x5=25개의 모델을 훈련)
# n_jobs는 병렬 실행에 사용할 CPU 코어 수를 지정. 기본값은 -1(모든 코어 사용)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [13]:
# 해당 모델을 일반 결정 트리처럼 사용 가능
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [14]:
# 최적의 매개변수는 best_params_ 에 저장됨
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [15]:
# 교차 검증의 평균 점수는 cv_results_의 mean_test_score에 저장되어 있음
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
# 과정 정리하여 다시 해보기
params = {
    'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
    'max_depth' : range(5, 20, 1),
    'min_samples_split' : range(2, 100, 10)
}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


## **랜덤 서치(Random Search)**

- 매개변수 값의 목록을 전달하지 않고, 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달함
- 연속된 매개변수 값을 탐색할 때 유용
- randint와 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 할 수 있음
  - 샘플링 횟수는 시스템 자원이 허락하는 선에서 최대한 크게 해보는 것이 좋음
- RandomizedSearchCV에 존재
  - n_iter : 샘플링 횟수 지정

In [23]:
from scipy.stats import uniform, randint
# randint는 정수값을 뽑는다
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 8, 0, 8, 6, 6, 6, 1, 9, 3])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([112,  92, 116, 102, 101, 100,  93,  82, 108,  94]))

In [25]:
# 0 ~ 1사이에서 10개의 실수 추출
# uniform은 실수값을 뽑는다
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.84682967, 0.90791673, 0.49604391, 0.36046446, 0.4772218 ,
       0.71847307, 0.03974041, 0.44590328, 0.14904463, 0.72451359])

In [26]:
# 이 개념을 이용하여 매개변수 탐색
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25),
}

In [27]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)


In [28]:
#최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [29]:
# 최종 모델로 테스트 세트 성능 검즘
# 테스트 세트 점수는 검증 세트 점수보다 조금 작은 것이 일반적
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
